In [1]:
import os
import folium
import psycopg2
import geopandas as gpd

# Connect to BC VRI database for inventory region 9 (Vancouver area)
conn = psycopg2.connect(database="cas", user="postgres", password="postgres")
y = gpd.read_postgis("select * from bc.vri9", conn, geom_col='wkb_geometry')

In [2]:
# Select stands > 600 years old
y500 = y[y['proj_age_1']>=500]
v500 = y500[['feature_id','wkb_geometry']]
d500 = y500[['feature_id','species_cd_1','proj_age_1','species_pct_1','species_cd_2','proj_age_2','species_pct_2']]
d500.head()

,feature_id,species_cd_1,proj_age_1,species_pct_1,species_cd_2,proj_age_2,species_pct_2
689,6123155,CW,521.0,40.0,HW,421.0,30.0
722,6266416,YC,521.0,80.0,HW,271.0,20.0
862,2417036,CW,521.0,80.0,HW,321.0,10.0
939,14032595,CW,521.0,70.0,HW,321.0,20.0
1298,2437263,BA,611.0,63.0,HW,601.0,32.0


In [6]:
v500.to_file("data/vri9.geojson",driver="GeoJSON")
j500 = gpd.read_file("data/vri9.geojson")
j500.head()

,feature_id,geometry
0,6123155,(POLYGON ((1230618.453763964 495049.5164924413...
1,6266416,(POLYGON ((1247368.255609604 498848.2242306415...
2,2417036,"(POLYGON ((1237526.74728786 492307.8676813897,..."
3,14032595,(POLYGON ((1245645.376728341 518577.1158709694...
4,2437263,(POLYGON ((1210694.002137115 502662.6809677575...


In [7]:
m = folium.Map(location=[49.5, -122.8], zoom_start=10, tiles="cartodbpositron")

m.choropleth(
    geo_data=j500,
    name='BC VRI',
    data=d500,
    columns=['species_cd_1', 'proj_age_1'],
    key_on='feature_id',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Projected age (yrs)')

folium.LayerControl().add_to(m)

#m.save("index.html")

m

RuntimeError: b'no arguments in initialization list'